# Download Sample RedPajama

togethercomputer/RedPajama-Data-1T-Sample

In [1]:
import os
os.getcwd()

'/data/home/milinbhade/Milin/AMC/bertamc_v4/finetune'

from datasets import load_dataset

dataset = load_dataset("togethercomputer/RedPajama-Data-1T-Sample", cache_dir='./redpajama')

In [17]:
data = load_dataset("yahma/alpaca-cleaned")

{'instruction': 'Give three tips for staying healthy.',
 'input': '',
 'output': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.'}

In [20]:
dataset = load_dataset("togethercomputer/RedPajama-Data-1T-Sample", cache_dir='./redpajama')

/data/home/milinbhade/miniconda3/envs/huggingface/lib/python3.9/site-packages/datasets/load.py:1461: FutureWarning: The repository for togethercomputer/RedPajama-Data-1T-Sample contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/togethercomputer/RedPajama-Data-1T-Sample
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [23]:
dataset['train']['text'][0]

"\\section{Introduction}\n\\label{sec:intro}\n\n\\emph{Gender diversity}, or more often its lack thereof, among participants to\nsoftware development activities has been thoroughly studied in recent years. In\nparticular, the presence of, effects of, and countermeasures for \\emph{gender\n  bias} in Free/Open Source Software (FOSS) have received a lot of attention\nover the past decade~\\cite{david2008fossdevs, qiu2010kdewomen,\n  nafus2012patches, kuechler2012genderfoss, vasilescu2014gender,\n  oneil2016debiansurvey, robles2016womeninfoss, terrell2017gender,\n  zacchiroli2021gender}.  \\emph{Geographic diversity} is on the other hand the\nkind of diversity that stems from participants in some global activity coming\nfrom different world regions and cultures.\n\nGeographic diversity in FOSS has received relatively little attention in scholarly\nworks. In particular, while seminal survey-based and\npoint-in-time medium-scale studies of the geographic origins of FOSS\ncontributors exist~

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
model_path = "/data/home/milinbhade/LLaMa/hf_llama/"
hf_llama_chat_path = "/data/home/milinbhade/LLaMa/hf_llama_chat/"

datapath = "/data/home/milinbhade/LLaMa/pretrain_data/alpaca/alpaca-cleaned-dataset"

# Lora pretrain

In [4]:
import os
import sys
from typing import List

import fire
import torch
import transformers
from datasets import load_dataset

In [5]:
torch.cuda.device_count()

1

In [6]:
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training,
    set_peft_model_state_dict,
)

In [7]:
from transformers import LlamaForCausalLM, LlamaTokenizer

In [8]:
import json
import os.path as osp
from typing import Union


class Prompter(object):
    __slots__ = ("template", "_verbose")

    def __init__(self, template_name: str = "", verbose: bool = False):
        self._verbose = verbose
        if not template_name:
            # Enforce the default here, so the constructor can be called with '' and will not break.
            template_name = "alpaca"
        file_name = osp.join("../templates", f"{template_name}.json")
        if not osp.exists(file_name):
            raise ValueError(f"Can't read {file_name}")
        with open(file_name) as fp:
            self.template = json.load(fp)
        if self._verbose:
            print(
                f"Using prompt template {template_name}: {self.template['description']}"
            )

    def generate_prompt(
        self,
        instruction: str,
        input: Union[None, str] = None,
        label: Union[None, str] = None,
    ) -> str:
        # returns the full prompt from instruction and optional input
        # if a label (=response, =output) is provided, it's also appended.
        if input:
            res = self.template["prompt_input"].format(
                instruction=instruction, input=input
            )
        else:
            res = self.template["prompt_no_input"].format(
                instruction=instruction
            )
        if label:
            res = f"{res}{label}"
        if self._verbose:
            print(res)
        return res

    def get_response(self, output: str) -> str:
        return output.split(self.template["response_split"])[1].strip()
    
class ZeroPrompter(object):
    __slots__ = ("_verbose")

    def __init__(self, verbose: bool = False):
        self._verbose = verbose
        
        if self._verbose:
            print(
                f"Without using prompt template!"
            )

    def generate_prompt(
        self,
        instruction: str,
        input: Union[None, str] = None,
        label: Union[None, str] = None,
    ) -> str:
        # returns the full prompt from instruction and optional input
        # if a label (=response, =output) is provided, it's also appended.
        if instruction[-1] == '.':
            instruction = instruction[:-1] + ':'
        if instruction[-1] not in ['.', ':', '?', '!']:
            instruction = instruction + ':'
        instruction += ' '

        if input:
            if input[-1] not in ['.', ':', '?', '!']:
                input = input + '.'
            res = instruction + input
        else:
            res = instruction
        if label:
            res = f"{res} {label}"
        if self._verbose:
            print(res)
        return res

    def get_response(self, output: str) -> str:
        return output.strip()

In [9]:
instruction = "Tell me a joke"
input_text = "Why did the chicken cross the road"
label = "To get to the other side"

prompter = Prompter(verbose=True)
print(prompter.generate_prompt(instruction, input_text, label))

output = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Tell me a joke

### Input:
Why did the chicken cross the road

### Response:
To get to the other side """
print("Get : ", prompter.get_response(output))

Using prompt template alpaca: Template used by Alpaca-LoRA.
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Tell me a joke

### Input:
Why did the chicken cross the road

### Response:
To get to the other side
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Tell me a joke

### Input:
Why did the chicken cross the road

### Response:
To get to the other side
Get :  To get to the other side


In [10]:
class Arguments:
    def __init__(self):
        self.base_model = model_path
        self.prune_model = None
        self.data_path = datapath
        self.cache_dataset = True
        self.extra_val_dataset = None
        self.output_dir = "./lora-alpaca"
        
        self.batch_size = 64
        self.micro_batch_size = 4
        self.num_epochs = 3
        self.learning_rate = 3e-4
        self.cutoff_len = 256
        self.val_set_size = 0.1
        self.prompt_template_name = "alpaca"
        self.no_instruction = False
        
        self.lora_r = 8
        self.lora_alpha = 16
        self.lora_dropout = 0.05
        self.lora_target_modules = "q_proj,k_proj,v_proj,o_proj,gate_proj,down_proj,up_proj"
        self.train_on_inputs = False
        self.add_eos_token = False
        self.group_by_length = False
        
        self.wandb_project = ""
        self.resume_from_checkpoint = None
        self.local_rank = -1
        self.torch_version = int(torch.__version__.split('.')[1])

args = Arguments()


In [11]:
def train(
    # model/data params
    base_model: str = model_path,  # the only required argument
    data_path: str = datapath,
    output_dir: str = "./lora-alpaca",
    # training hyperparams
    batch_size: int = 128,
    micro_batch_size: int = 4,
    num_epochs: int = 3,
    learning_rate: float = 3e-4,
    cutoff_len: int = 256,
    val_set_size: int = 0.1,
    # lora hyperparams
    lora_r: int = 8,
    lora_alpha: int = 16,
    lora_dropout: float = 0.05,
    lora_target_modules: List[str] = [
        "q_proj","k_proj","v_proj","o_proj","gate_proj","down_proj","up_proj"
    ],
    # llm hyperparams
    train_on_inputs: bool = True,  # if False, masks out inputs in loss
    add_eos_token: bool = False,
    group_by_length: bool = False,  # faster, but produces an odd training loss curve
    # wandb params
    wandb_project: str = "",
    wandb_run_name: str = "",
    wandb_watch: str = "",  # options: false | gradients | all
    wandb_log_model: str = "",  # options: false | true
    resume_from_checkpoint: str = None,  # either training checkpoint or final adapter
    prompt_template_name: str = "alpaca",  # The prompt template to use, will default to alpaca.
):
    if int(os.environ.get("LOCAL_RANK", 0)) == 0:
        print(
            f"Training Alpaca-LoRA model with params:\n"
            f"base_model: {base_model}\n"
            f"data_path: {data_path}\n"
            f"output_dir: {output_dir}\n"
            f"batch_size: {batch_size}\n"
            f"micro_batch_size: {micro_batch_size}\n"
            f"num_epochs: {num_epochs}\n"
            f"learning_rate: {learning_rate}\n"
            f"cutoff_len: {cutoff_len}\n"
            f"val_set_size: {val_set_size}\n"
            f"lora_r: {lora_r}\n"
            f"lora_alpha: {lora_alpha}\n"
            f"lora_dropout: {lora_dropout}\n"
            f"lora_target_modules: {lora_target_modules}\n"
            f"train_on_inputs: {train_on_inputs}\n"
            f"add_eos_token: {add_eos_token}\n"
            f"group_by_length: {group_by_length}\n"
            f"wandb_project: {wandb_project}\n"
            f"wandb_run_name: {wandb_run_name}\n"
            f"wandb_watch: {wandb_watch}\n"
            f"wandb_log_model: {wandb_log_model}\n"
            f"resume_from_checkpoint: {resume_from_checkpoint or False}\n"
            f"prompt template: {prompt_template_name}\n"
        )
    assert (
        base_model
    ), "Please specify a --base_model, e.g. --base_model='huggyllama/llama-7b'"
    gradient_accumulation_steps = batch_size // micro_batch_size

    prompter = Prompter(prompt_template_name)

    device_map = "auto"
    world_size = int(os.environ.get("WORLD_SIZE", 1))
    ddp = world_size != 1
    if ddp:
        device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}
        gradient_accumulation_steps = gradient_accumulation_steps // world_size

    # Check if parameter passed or if set within environ
    use_wandb = len(wandb_project) > 0 or (
        "WANDB_PROJECT" in os.environ and len(os.environ["WANDB_PROJECT"]) > 0
    )
    # Only overwrite environ if wandb param passed
    if len(wandb_project) > 0:
        os.environ["WANDB_PROJECT"] = wandb_project
    if len(wandb_watch) > 0:
        os.environ["WANDB_WATCH"] = wandb_watch
    if len(wandb_log_model) > 0:
        os.environ["WANDB_LOG_MODEL"] = wandb_log_model

    model = LlamaForCausalLM.from_pretrained(
        base_model,
        load_in_8bit=True,
        torch_dtype=torch.bfloat16,
        device_map=device_map,
    )

    tokenizer = LlamaTokenizer.from_pretrained(base_model)

    tokenizer.pad_token_id = (
        0  # unk. we want this to be different from the eos token
    )
    tokenizer.padding_side = "left"  # Allow batched inference

    def tokenize(prompt, add_eos_token=True):
        # there's probably a way to do this with the tokenizer settings
        # but again, gotta move fast
        result = tokenizer(
            prompt,
            truncation=True,
            max_length=cutoff_len,
            padding=False,
            return_tensors=None,
        )
        if (
            result["input_ids"][-1] != tokenizer.eos_token_id
            and len(result["input_ids"]) < cutoff_len
            and add_eos_token
        ):
            result["input_ids"].append(tokenizer.eos_token_id)
            result["attention_mask"].append(1)

        result["labels"] = result["input_ids"].copy()

        return result

    def generate_and_tokenize_prompt(data_point):
        full_prompt = prompter.generate_prompt(
            data_point["instruction"],
            data_point["input"],
            data_point["output"],
        )
        tokenized_full_prompt = tokenize(full_prompt)
        if not train_on_inputs:
            user_prompt = prompter.generate_prompt(
                data_point["instruction"], data_point["input"]
            )
            tokenized_user_prompt = tokenize(
                user_prompt, add_eos_token=add_eos_token
            )
            user_prompt_len = len(tokenized_user_prompt["input_ids"])

            if add_eos_token:
                user_prompt_len -= 1

            tokenized_full_prompt["labels"] = [
                -100
            ] * user_prompt_len + tokenized_full_prompt["labels"][
                user_prompt_len:
            ]  # could be sped up, probably
        return tokenized_full_prompt

    model = prepare_model_for_kbit_training(model)

    config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        target_modules=lora_target_modules,
        lora_dropout=lora_dropout,
        bias="none",
        task_type="CAUSAL_LM",
    )
    model = get_peft_model(model, config)

    if data_path.endswith(".json") or data_path.endswith(".jsonl"):
        print("If")
        data = load_dataset("json", data_files=data_path)
        print(data)
    else:
        data = load_dataset("yahma/alpaca-cleaned")

        print("Else")
        print(data)
    if resume_from_checkpoint:
        # Check the available weights and load them
        checkpoint_name = os.path.join(
            resume_from_checkpoint, "pytorch_model.bin"
        )  # Full checkpoint
        if not os.path.exists(checkpoint_name):
            checkpoint_name = os.path.join(
                resume_from_checkpoint, "adapter_model.bin"
            )  # only LoRA model - LoRA config above has to fit
            resume_from_checkpoint = (
                False  # So the trainer won't try loading its state
            )
        # The two files above have a different name depending on how they were saved, but are actually the same.
        if os.path.exists(checkpoint_name):
            print(f"Restarting from {checkpoint_name}")
            adapters_weights = torch.load(checkpoint_name)
            set_peft_model_state_dict(model, adapters_weights)
        else:
            print(f"Checkpoint {checkpoint_name} not found")

    model.print_trainable_parameters()  # Be more transparent about the % of trainable params.

    if val_set_size > 0:
        train_val = data["train"].train_test_split(
            test_size=val_set_size, shuffle=True, seed=42
        )
        train_data = (
            train_val["train"].shuffle().map(generate_and_tokenize_prompt)
        )
        val_data = (
            train_val["test"].shuffle().map(generate_and_tokenize_prompt)
        )
    else:
        train_data = data["train"].shuffle().map(generate_and_tokenize_prompt)
        val_data = None

    if not ddp and torch.cuda.device_count() > 1:
        # keeps Trainer from trying its own DataParallelism when more than 1 gpu is available
        model.is_parallelizable = True
        model.model_parallel = True

    trainer = transformers.Trainer(
        model=model,
        train_dataset=train_data,
        eval_dataset=val_data,
        args=transformers.TrainingArguments(
            per_device_train_batch_size=micro_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            warmup_steps=100,
            num_train_epochs=num_epochs,
            learning_rate=learning_rate,
            fp16=True,
            logging_steps=10,
            optim="adamw_torch",
            evaluation_strategy="steps" if val_set_size > 0 else "no",
            save_strategy="steps",
            eval_steps=200 if val_set_size > 0 else None,
            save_steps=200,
            output_dir=output_dir,
            save_total_limit=3,
            load_best_model_at_end=True if val_set_size > 0 else False,
            ddp_find_unused_parameters=False if ddp else None,
            group_by_length=group_by_length,
            report_to="wandb" if use_wandb else None,
            run_name=wandb_run_name if use_wandb else None,
        ),
        data_collator=transformers.DataCollatorForSeq2Seq(
            tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
        ),
    )
    model.config.use_cache = False

    old_state_dict = model.state_dict
    model.state_dict = (
        lambda self, *_, **__: get_peft_model_state_dict(
            self, old_state_dict()
        )
    ).__get__(model, type(model))

    if torch.__version__ >= "2" and sys.platform != "win32":
        model = torch.compile(model)

    trainer.train(resume_from_checkpoint=resume_from_checkpoint)

    model.save_pretrained(output_dir)

    print(
        "\n If there's a warning about missing keys above, please disregard :)"
    )


In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling
from transformers import BitsAndBytesConfig
device = "cuda" if torch.cuda.is_available() else "cpu"

2024-05-29 01:32:57.999837: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-29 01:32:59.029716: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


from transformers import AutoPeftModelForCausalLM

In [13]:
device

'cuda'

In [14]:
from trl import SFTTrainer

In [15]:
from pathlib import Path
def main(args):
    # Set WanDB
    os.environ["WANDB_PROJECT"] = args.wandb_project
    
    quantization_config = BitsAndBytesConfig(
        load_in_8bit=True,
#         bnb_4bit_use_double_quant=True,
#         bnb_4bit_quant_type="nf4",
        bnb_8bit_compute_dtype=torch.bfloat16,
    )
    
    # Load Pruned Model
    model = model = AutoModelForCausalLM.from_pretrained(
        args.base_model,  
        torch_dtype=torch.bfloat16, 
        quantization_config=quantization_config,
        low_cpu_mem_usage=True, 
        device_map="auto"
    )
    
    model.seqlen = model.config.max_position_embeddings 
    print(model.seqlen)
    
    
    
    
    tokenizer= LlamaTokenizer.from_pretrained(args.base_model,padding=True, truncation=True)

    gradient_accumulation_steps = args.batch_size // args.micro_batch_size
    if not args.no_instruction:
        prompter = Prompter(args.prompt_template_name)
    else:
        prompter = ZeroPrompter()

    world_size = int(os.environ.get("WORLD_SIZE", 1))
    ddp = world_size != 1
    if ddp:
        gradient_accumulation_steps = gradient_accumulation_steps // world_size

#     if device == 'cuda':
#         model.half()

    tokenizer.pad_token_id = 0
    tokenizer.padding_side = "left"

    def tokenize(prompt, add_eos_token=True):
        result = tokenizer(
            prompt,
            truncation=True,
            max_length=args.cutoff_len,
            padding=False,
            return_tensors=None,
        )
        if (
            result["input_ids"][-1] != tokenizer.eos_token_id
            and len(result["input_ids"]) < args.cutoff_len
            and add_eos_token
        ):
            result["input_ids"].append(tokenizer.eos_token_id)
            result["attention_mask"].append(1)

        result["labels"] = result["input_ids"].copy()

        return result

    def generate_and_tokenize_prompt(data_point):
        if 'lamini' in args.data_path.lower():
            full_prompt = prompter.generate_prompt(
                data_point["instruction"],
                None,
                data_point["response"],
            )
        elif 'alpaca' in args.data_path.lower():
            full_prompt = prompter.generate_prompt(
                data_point["instruction"],
                data_point["input"],
                data_point["output"],
            )
        else:
            raise NotImplementedError

        tokenized_full_prompt = tokenize(full_prompt)
        if not args.train_on_inputs:
            user_prompt = prompter.generate_prompt(
                data_point["instruction"], data_point["input"] if 'input' in data_point.keys() else None,
            )
            tokenized_user_prompt = tokenize(
                user_prompt, add_eos_token=args.add_eos_token
            )
            user_prompt_len = len(tokenized_user_prompt["input_ids"])

            if args.add_eos_token:
                user_prompt_len -= 1

            tokenized_full_prompt["labels"] = [
                -100
            ] * user_prompt_len + tokenized_full_prompt["labels"][
                user_prompt_len:
            ]  # could be sped up, probably
        return tokenized_full_prompt

    def split_and_tokenizer(test_data, tokenizer, seq_len, field_name):
        test_ids = tokenizer("\n\n".join(test_data[field_name]), return_tensors='pt').input_ids[0]
        nsamples = test_ids.numel() // seq_len

        test_set = []
        for i in range(nsamples):
            batch = test_ids[(i * seq_len):((i + 1) * seq_len)]
            test_set.append({
                'input_ids': batch,
                'labels': batch
            })
        return test_set

    
    model.gradient_checkpointing_enable()
    # Prepare For LoRA
    model = prepare_model_for_kbit_training(model)
    
    
    config = LoraConfig(
        r=args.lora_r,
        lora_alpha=args.lora_alpha,
        target_modules=args.lora_target_modules.split(","),
        lora_dropout=args.lora_dropout,
        bias="none",
        task_type="CAUSAL_LM",
    )
    model = get_peft_model(model, config)
    model.print_trainable_parameters()  

    # Load Train Dataset
    data = load_dataset("yahma/alpaca-cleaned")
    
    print("Data Length: ", len(data))
    print(data)
    
    
    
    
    if args.cache_dataset and os.path.exists('./datasets/cache/{}.bin'.format("alpaca")):
        print("Entered If")
        preprocess_data = torch.load('./datasets/cache/{}.bin'.format("alpaca"))
        train_data, val_data = preprocess_data['train'], preprocess_data['val']
    else:
        print("Entered Else")
        train_val = data["train"].train_test_split(
            test_size=args.val_set_size, shuffle=True, seed=42
        )
        train_data = (
            train_val["train"].shuffle().map(generate_and_tokenize_prompt)
        )
        val_data = {
            args.data_path: train_val["test"].shuffle().map(generate_and_tokenize_prompt),
        }
        if args.cache_dataset and args.local_rank == -1:
            cache_file = './datasets/cache/{}.bin'.format("alpaca")
            cache_dir = '/'.join(cache_file.split('/')[:-1])
            directory = Path(cache_dir)
            directory.mkdir(parents=True, exist_ok=True)
            print("Saving datasets")
            torch.save({
                'train': train_data, 'val': val_data
            }, cache_file)

    # Load Extra Validation Dataset
    if args.extra_val_dataset:
        from LLMPruner.datasets.ppl_dataset import get_wikitext2, get_ptb

        seq_len = 128
        for extra_dataset in args.extra_val_dataset.split(','):
            if 'wikitext2' in extra_dataset:
                _, test_data = get_wikitext2(seq_len, None)
                test_data = split_and_tokenizer(test_data, tokenizer, seq_len, field_name='text')
            if 'ptb' in extra_dataset:
                _, test_data = get_ptb(seq_len, None)
                test_data = split_and_tokenizer(test_data, tokenizer, seq_len, field_name='sentence')
            val_data[extra_dataset] = test_data

#     trainer = transformers.Trainer(
#         model=model,
#         train_dataset=train_data,
#         eval_dataset=val_data,
#         args=transformers.TrainingArguments(
#             per_device_train_batch_size=args.micro_batch_size,
#             gradient_accumulation_steps=gradient_accumulation_steps,
#             warmup_steps=10,
#             num_train_epochs=args.num_epochs,
#             learning_rate=args.learning_rate,
#             fp16=True,
#             logging_steps=10,
#             logging_first_step=True,
#             optim="adamw_torch",
#             evaluation_strategy="steps",
#             save_strategy="steps",
#             eval_steps=100,
#             save_steps=200,
#             output_dir=args.output_dir,
#             save_total_limit=20,
#             load_best_model_at_end=True,
#             ddp_find_unused_parameters=None,
#             group_by_length=args.group_by_length,
#             report_to="wandb",
#             run_name=args.output_dir.split('/')[-1],
#             metric_for_best_model="{}_loss".format(args.data_path),
#         ),
# #         data_collator=transformers.DataCollatorForSeq2Seq(
# #             tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
# #         ),
#         data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
#     )





#     trainer = SFTTrainer(
#         model=model,
#         train_dataset=train_data,
# #         eval_dataset=val_data,
#         args=transformers.TrainingArguments(
#             per_device_train_batch_size=8,
#             gradient_accumulation_steps=8,
#             warmup_steps=10,
#             num_train_epochs=3,
#             learning_rate=2e-4,
#             fp16=True,
#             logging_steps=10,
#             optim="paged_adamw_8bit",
#             output_dir=args.output_dir,
#             report_to="wandb",
#             run_name=args.output_dir.split('/')[-1],
#         ),
#         data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
#     )
    
    trainer = SFTTrainer(
        model=model,
        train_dataset=train_data,
        eval_dataset=val_data,  # Pass validation dataset here
        peft_config=config,
        dataset_text_field="input_ids",
        max_seq_length=1024,
        tokenizer=tokenizer,
        args=transformers.TrainingArguments(
            per_device_train_batch_size=args.micro_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            warmup_steps=10,
            num_train_epochs=args.num_epochs,
            learning_rate=args.learning_rate,
            fp16=True,
            logging_steps=10,
            logging_first_step=True,
            optim="adamw_torch",
            evaluation_strategy="steps",
            save_strategy="steps",
            eval_steps=100,
            save_steps=200,
            output_dir=args.output_dir,
            save_total_limit=20,
            load_best_model_at_end=True,
            ddp_find_unused_parameters=None,
            group_by_length=args.group_by_length,
            report_to="wandb",
            run_name=args.output_dir.split('/')[-1],
            metric_for_best_model="{}_loss".format(args.data_path),
        ),
        packing=False,
    )
    
    
    model.config.use_cache = False
    model.config.pretraining_tp = 1
    
    
    dtypes = {}
    for _, p in model.named_parameters():
        dtype = p.dtype
        if dtype not in dtypes: dtypes[dtype] = 0
        dtypes[dtype] += p.numel()
    total = 0
    for k, v in dtypes.items(): total+= v
    for k, v in dtypes.items():
        print(k, v, v/total)
        
        
        
#     old_state_dict = model.state_dict
#     model.state_dict = (
#         lambda self, *_, **__: get_peft_model_state_dict(
#             self, old_state_dict()
#         )
#     ).__get__(model, type(model))
    
    
    
    with torch.autocast("cuda"): 
        train_result = trainer.train(resume_from_checkpoint=args.resume_from_checkpoint)
        metrics = train_result.metrics
        trainer.log_metrics("train", metrics)
        trainer.save_metrics("train", metrics)
        trainer.save_state()
        print(metrics)

    
    
#     model.state_dict = old_state_dict
#     model.save_pretrained(args.output_dir)

    # Saving model
    print("Completed training with lora alpha : ",args.lora_alpha," and device : 4. Storage location : ",args.output_dir)
    print("Saving last checkpoint of the model...")
    final_ckpt = os.path.join(args.output_dir, "checkpoint")
    os.makedirs(final_ckpt, exist_ok=True)
    trainer.model.save_pretrained(final_ckpt)
    print("Last Checkpoint saved successfully")
    # Free memory for merging weights
    del model
    del trainer
    torch.cuda.empty_cache()
    
    
    
#     print("Merging models")
#     model = AutoPeftModelForCausalLM.from_pretrained(output_dir_final_cp, device_map="auto", torch_dtype=torch.bfloat16)
#     model = model.merge_and_unload()
#     print("Model Merged")
    
    
    
#     model.save_pretrained(output_dir_merged, safe_serialization=True)
#     tokenizer.save_pretrained(output_dir_merged)
#     print("Merged Model saved successfully")

#     print("Training Finished")

In [16]:
main(args)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/data/home/milinbhade/miniconda3/envs/huggingface/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


2048
trainable params: 19,988,480 || all params: 6,758,404,096 || trainable%: 0.2957573965106688


/data/home/milinbhade/miniconda3/envs/huggingface/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Data Length:  1
DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 51760
    })
})
Entered If
torch.float32 282398720 0.04178482316071323
torch.int8 6476005376 0.9582151768392868


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: milinbhade1214 (milinbhade1). Use `wandb login --relogin` to force relogin


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`labels` in this case) have excessive nesting (inputs type `list` where type `int` is expected).